In [2]:
!pip install rogue

In [13]:
## for data
import pandas as pd
import numpy as np

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for analysis
import re
import nltk
import contractions

## for machine learning
from sklearn import preprocessing, model_selection, feature_extraction, feature_selection, metrics, manifold, naive_bayes, pipeline
from sklearn.model_selection import train_test_split

## for deep learning
from tensorflow.keras import callbacks, models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input,LSTM,Embedding,Dense,Concatenate,Attention
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

## for W2V and textRank
import gensim
import gensim.downloader as gensim_api

## for summarization
#import rouge
import difflib

## for time
import time

In [2]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    print("Physical Devices : {}".format(tf.config.list_physical_devices()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0
Physical Devices : [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-03-31 16:52:17.821096: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 16:52:23.864034: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 30998 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2023-03-31 16:52:23.898129: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 30998 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


<h2> Data Reading and Preprocessing </h2>

In [3]:
#FUNCTION TO CREATE STOP WORDS
def create_stopwords(lst_langs=["english"], lst_add_words=[], lst_keep_words=[]):
    lst_stopwords = set()
    for lang in lst_langs:
        lst_stopwords = lst_stopwords.union( set(nltk.corpus.stopwords.words(lang)) )
    lst_stopwords = lst_stopwords.union(lst_add_words)
    lst_stopwords = list(set(lst_stopwords) - set(lst_keep_words))
    return sorted(list(set(lst_stopwords)))

In [4]:
lst_stopwords = create_stopwords()
print(len(lst_stopwords))

179


In [5]:
df = pd.read_csv("Wikipedia-Level-4-Articles.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,Title,Article,Category,Summary
0,Masaccio,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De...",People,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De..."
1,Hokusai,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ...",People,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ..."
2,Steven Spielberg,"Steven Allan Spielberg (; born December 18, 1...",People,"Steven Allan Spielberg (; born December 18, 1..."
3,Che Guevara,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]...",People,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]..."
4,Jöns Jacob Berzelius,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...,People,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...
...,...,...,...,...
10004,Zimbabwe,"Zimbabwe (), officially the Republic of Zimbab...",Geography,"Zimbabwe (), officially the Republic of Zimbab..."
10005,Nile,The Nile is a major north-flowing river in nor...,Geography,The Nile is a major north-flowing river in nor...
10006,Republic of Ireland,"Ireland (Irish: Éire [ˈeːɾʲə] (listen)), also ...",Geography,"Ireland (Irish: Éire [ˈeːɾʲə] (listen)), also ..."
10007,Lake Erie,"Lake Erie ( ""eerie"") is the fourth largest lak...",Geography,"Lake Erie ( ""eerie"") is the fourth largest lak..."


In [6]:
#FUNCTION TO CLEAN THE TEXT

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
def utils_preprocess_text(txt, lst_regex=None, punkt=True, lower=True, slang=True, lst_stopwords=None, stemm=False, lemm=True):
    ## Regex (in case, before cleaning)
    if lst_regex is not None: 
        for regex in lst_regex:
            txt = re.sub(regex, '', txt)

    ## Clean 
    ### separate sentences with '. '
    txt = re.sub(r'\.(?=[^ \W\d])', '. ', str(txt))
    ### remove punctuations and characters
    txt = re.sub(r'[^\w\s]', '', txt) if punkt is True else txt
    ### strip
    txt = " ".join([word.strip() for word in txt.split()])
    ### lowercase
    txt = txt.lower() if lower is True else txt
    ### slang
    txt = contractions.fix(txt) if slang is True else txt
            
    ## Tokenize (convert from string to list)
    lst_txt = txt.split()
                
    ## Stemming (remove -ing, -ly, ...)
    if stemm is True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_txt = [ps.stem(word) for word in lst_txt]
                
    ## Lemmatization (convert the word into root word)
    if lemm is True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_txt = [lem.lemmatize(word) for word in lst_txt]

    ## Stopwords
    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in lst_stopwords]
        
    lst_txt = [ word for word in lst_txt if isEnglish(word) is True]
    
    # remove words less than three letters
    lst_txt = [word for word in lst_txt if len(word) >= 3]
            
    ## Back to string
    txt = " ".join(lst_txt)
    return txt



#FUNCTION TO ADD A COLUMN CONTAINING THE CLEANED TEXT
def add_preprocessed_text(data, column, lst_regex=None, punkt=False, lower=False, slang=False, lst_stopwords=None, stemm=False, lemm=True, remove_na=True):
    dtf = data.copy()

    ## apply preprocess
    dtf = dtf[ pd.notnull(dtf[column]) ]
    dtf[column+"_clean"] = dtf[column].apply(lambda x: utils_preprocess_text(x, lst_regex, punkt, lower, slang, lst_stopwords, stemm, lemm))
    
    ## residuals
    dtf["check"] = dtf[column+"_clean"].apply(lambda x: len(x))
    if dtf["check"].min() == 0:
        print("--- found NAs ---")
        print(dtf[[column,column+"_clean"]][dtf["check"]==0].head())
        if remove_na is True:
            dtf = dtf[dtf["check"]>0] 
            
    return dtf.drop("check", axis=1)

In [7]:
print("Starting to clean the Articles")
start = time.time()
df_cleaned = add_preprocessed_text(df, column="Article", 
                            punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, lemm=True,stemm=True)
end = time.time()
print("Time take to clean the Articles : {}".format(end-start))


Starting to clean the Articles
Time take to clean the Articles : 860.2032513618469


In [8]:
print("Starting to clean the Summary")
start = time.time()
df_cleaned = add_preprocessed_text(df_cleaned, column="Summary", 
                            punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, lemm=True)
end = time.time()
print("Time take to clean the Summary : {}".format(end-start))


Starting to clean the Summary
Time take to clean the Summary : 12.686798572540283


In [9]:
df_cleaned

,Title,Article,Category,Summary,Article_clean,Summary_clean
0,Masaccio,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De...",People,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De...",masaccio italian decemb 1401 summer 1428 born ...,masaccio italian december 1401 summer 1428 bor...
1,Hokusai,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ...",People,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ...",katsushika hokusai octob 1760 may 1849 known s...,katsushika hokusai october 1760 may 1849 known...
2,Steven Spielberg,"Steven Allan Spielberg (; born December 18, 1...",People,"Steven Allan Spielberg (; born December 18, 1...",steven allan spielberg born decemb 1946 americ...,steven allan spielberg born december 1946 amer...
3,Che Guevara,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]...",People,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]...",ernesto che guevara spanish june 1928 octob 19...,ernesto che guevara spanish june 1928 october ...
4,Jöns Jacob Berzelius,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...,People,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...,baron jacob berzeliu swedish contemporari name...,baron jacob berzelius swedish contemporary nam...
...,...,...,...,...,...,...
10004,Zimbabwe,"Zimbabwe (), officially the Republic of Zimbab...",Geography,"Zimbabwe (), officially the Republic of Zimbab...",zimbabw offici republ zimbabw landlock countri...,zimbabwe officially republic zimbabwe landlock...
10005,Nile,The Nile is a major north-flowing river in nor...,Geography,The Nile is a major north-flowing river in nor...,nile major northflow river northeastern africa...,nile major northflowing river northeastern afr...
10006,Republic of Ireland,"Ireland (Irish: Éire [ˈeːɾʲə] (listen)), also ...",Geography,"Ireland (Irish: Éire [ˈeːɾʲə] (listen)), also ...",ireland irish listen also known republ ireland...,ireland irish listen also known republic irela...
10007,Lake Erie,"Lake Erie ( ""eerie"") is the fourth largest lak...",Geography,"Lake Erie ( ""eerie"") is the fourth largest lak...",lake eri eeri fourth largest lake surfac area ...,lake erie eerie fourth largest lake surface ar...


In [10]:
df_cleaned.to_csv("Wikipedia-Level-4-Articles-Cleaned-2.csv")

<h2>Cleaned DF</h2>

In [14]:
df_cleaned = pd.read_csv("Wikipedia-Level-4-Articles-Cleaned-1.csv")

# Define the categories you want to select
categories = ["People", "Arts", "History", "Biology_and_health_sciences", "Everyday_life", "Geography", "Philosophy_and_religion", "Society_and_social_sciences"]

# Create an empty list to store the selected rows
selected_rows = []

# Loop through each category and select 30 rows
for category in categories:
    category_rows = df_cleaned[df_cleaned["Category"] == category].head(200)
    selected_rows.append(category_rows)

# Concatenate all the selected rows into a new dataframe
df_cleaned = pd.concat(selected_rows)

In [15]:
df_cleaned

,Unnamed: 0,Title,Article,Category,Summary,Article_clean,Summary_clean
0,0,Masaccio,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De...",People,"Masaccio (UK: , US: , Italian: [maˈzattʃo]; De...",masaccio uk u italian december 21 1401 summer ...,masaccio uk u italian december 21 1401 summer ...
1,1,Hokusai,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ...",People,"Katsushika Hokusai (葛飾 北斎, c. 31 October 1760 ...",katsushika hokusai c 31 october 1760 10 may 18...,katsushika hokusai c 31 october 1760 10 may 18...
2,2,Steven Spielberg,"Steven Allan Spielberg (; born December 18, 1...",People,"Steven Allan Spielberg (; born December 18, 1...",steven allan spielberg born december 18 1946 a...,steven allan spielberg born december 18 1946 a...
3,3,Che Guevara,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]...",People,"Ernesto ""Che"" Guevara (Spanish: [ˈtʃe ɣeˈβaɾa]...",ernesto che guevara spanish 14 june 1928 9 oct...,ernesto che guevara spanish 14 june 1928 9 oct...
4,4,Jöns Jacob Berzelius,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...,People,Baron Jöns Jacob Berzelius (Swedish: [jœns ˈjɑ...,baron jacob berzelius swedish contemporary nam...,baron jacob berzelius swedish contemporary nam...
...,...,...,...,...,...,...,...
10004,10004,Zimbabwe,"Zimbabwe (), officially the Republic of Zimbab...",Geography,"Zimbabwe (), officially the Republic of Zimbab...",zimbabwe officially republic zimbabwe landlock...,zimbabwe officially republic zimbabwe landlock...
10005,10005,Nile,The Nile is a major north-flowing river in nor...,Geography,The Nile is a major north-flowing river in nor...,nile major northflowing river northeastern afr...,nile major northflowing river northeastern afr...
10006,10006,Republic of Ireland,"Ireland (Irish: Éire [ˈeːɾʲə] (listen)), also ...",Geography,"Ireland (Irish: Éire [ˈeːɾʲə] (listen)), also ...",ireland irish listen also known republic irela...,ireland irish listen also known republic irela...
10007,10007,Lake Erie,"Lake Erie ( ""eerie"") is the fourth largest lak...",Geography,"Lake Erie ( ""eerie"") is the fourth largest lak...",lake erie eerie fourth largest lake surface ar...,lake erie eerie fourth largest lake surface ar...
